In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("Data/qaly_input.csv")

In [3]:
to_fit

,Description,Value,Lower Bound,Upper Bound
0,Child-Ototoxicity,0.734,0.598,0.871
1,Child-NCD,0.520,0.300,0.750
2,Mother-Ototoxicity or NCD,0.850,0.740,0.940
3,Mother-Infant Death,0.920,0.600,0.990


In [4]:
# fits an encompassing beta distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_beta_distr(value, lower_bound, upper_bound):
    og_alpha = value
    og_beta = 1-og_alpha
    
    for scale in range(200):
        scale = 200 - scale
        alpha = og_alpha * scale
        beta = og_beta * scale
        ppf_025 = ss.beta.ppf(0.025, alpha, beta)
        ppf_975 = ss.beta.ppf(0.975, alpha, beta)
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            print("alpha: %.2f \t beta: %.2f" % (alpha, beta))           
            return alpha, beta
        
    print("No match found")
    return 0, 0

In [5]:
to_fit["alpha"], to_fit["beta"] = np.vectorize(fit_encompassing_beta_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

alpha: 22.75 	 beta: 8.25
alpha: 22.75 	 beta: 8.25
alpha: 8.32 	 beta: 7.68
alpha: 34.85 	 beta: 6.15
alpha: 4.60 	 beta: 0.40


In [6]:
to_fit["beta_mean"] = ss.beta.mean(to_fit["alpha"], to_fit["beta"])
to_fit["beta_2.5pct"] = ss.beta.ppf(0.025, to_fit["alpha"], to_fit["beta"])
to_fit["beta_97.5pct"] = ss.beta.ppf(0.975, to_fit["alpha"], to_fit["beta"])

In [7]:
to_fit

,Description,Value,Lower Bound,Upper Bound,alpha,beta,beta_mean,beta_2.5pct,beta_97.5pct
0,Child-Ototoxicity,0.734,0.598,0.871,22.754,8.246,0.734,0.568196,0.871262
1,Child-NCD,0.520,0.300,0.750,8.320,7.680,0.520,0.283688,0.751576
2,Mother-Ototoxicity or NCD,0.850,0.740,0.940,34.850,6.150,0.850,0.727356,0.940517
3,Mother-Infant Death,0.920,0.600,0.990,4.600,0.400,0.920,0.597570,0.999983


In [8]:
to_fit.to_csv("Data/qaly_output.csv")